In [1]:
%matplotlib inline

# Extending Sequence Prediction Using Character Level Recurrent Networks

We extend the approach outlined in `char_rnn_one_file_code_gen.ipynb` to improve efficiency and to accomodate other recurrent cells. Notably:

* Training will use multiple files, instead of a single file
* Validation sets will be introduced to avoid overfitting on our training data
* We will have our model utilize mini-batches to speed up training

Our outlined task is still the same.

**Given a sequence of characters, predict the next likely character in the sequence.**

In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
from json import load
from math import floor
from time import time

## Preparing the Data

We will be training on a set of 180 preprocessed Python files (and validating on a set of 20 other files) arbitrarily sampled from [GitHub BigQuery Python Extracts](https://bigquery.cloud.google.com/table/fh-bigquery:github_extracts.contents_py_201802snap?pli=1).

We limit the characters that our neural network can produce to a subset of standard ASCII.
* `ORD 2*, 3*, 9, 10, 32-126`
  * `ORD 0` for padding (special, used in batch_size > 1 with variable length sequences)
  * `ORD 2` for start of text (special, never predicted)
  * `ORD 3` for end of text (special, prediction ends)
  * `ORD 9` horizontal tab "\t"
  * `ORD 10` NL line feed, new line "\n"
  * `ORD 32-126` Space, Punctuation, Digits, English Letters

NOTE: The full dataset contains files written using non standard characters. For the models in this notebook, we ensure that all Python files within our dataset are composed only of ASCII characters that we accept.

In [3]:
# Possible Characters for neural network
VALID_UNICODE_IDS = (0, 2, 3, 9, 10) + tuple(range(32, 127))
for uid in VALID_UNICODE_IDS:
    if uid <= 32:
        print("{}: {}".format(uid, repr(chr(uid))))
        continue
    print(chr(uid), end="")
print()

# Special Characters
PAD = chr(0)
FILE_START = chr(2)
FILE_END = chr(3)

CHARACTERS = set(chr(id) for id in VALID_UNICODE_IDS)
INT2CHAR = dict(enumerate(CHARACTERS))
CHAR2INT = {char: idx for idx, char in INT2CHAR.items()}

with open("./data/train.json", "r") as f:
    training_data = load(f)
with open("./data/validate.json", "r") as f:
    validation_data = load(f)

print("~" * 25)
print("Num Training Files: {}".format(len(training_data)))
print("Num Validation Files: {}".format(len(validation_data)))

0: '\x00'
2: '\x02'
3: '\x03'
9: '\t'
10: '\n'
32: ' '
!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
~~~~~~~~~~~~~~~~~~~~~~~~~
Num Training Files: 180
Num Validation Files: 20


Let's randomly take a look at what is contained within our training data.

In [4]:
train_sample = random.sample(training_data, 1)[0]
print("".join(train_sample))

class dotdictify(dict):

    def __init__(self, value=None):
        if value is None:
            pass
        elif isinstance(value, dict):
            for key in value:
                self.__setitem__(key, value[key])
        else:
            raise TypeError('expected dict')

    def __setitem__(self, key, value):
        if '.' in key:
            myKey, restOfKey = key.split('.', 1)
            target = self.setdefault(myKey, dotdictify())
            if not isinstance(target, dotdictify) and isinstance(target, dict):
                target = dotdictify(target)
            if not isinstance(target, dotdictify):
                raise KeyError('cannot set "%s" in "%s" (%s)' % (restOfKey,
                    myKey, repr(target)))
            target[restOfKey] = value
        else:
            if isinstance(value, dict) and not isinstance(value, dotdictify):
                value = dotdictify(value)
            dict.__setitem__(self, key, value)

    def __getitem__(self, key):
   

## Batching Sliding Window Algorithm

Extending from the previously created sliding window algorithm:

* Given some iterable, we want a generator that yields X, Y pairs for evaluation.
* We want a sequence of a given context length as X, and the next character as Y.

We want our sliding window algorithm to yield these pairs in batches, for more efficient computation. Iterable is no longer a single file, but an iterator over multiple files.

In [5]:
def batch_sliding_window_generator(iterable_files, batch_size, max_window_size=None, gen_forever=True):
    """Sliding window generator for batching files of ASCII characters
    """
    while True:
        data_pairs = []

        for iterable in iterable_files:
            if max_window_size is None:
                window_size = len(iterable)
            else:
                window_size = max_window_size

            # edge case inbetween files
            existing_window_sizes = [len(x) for x, _ in data_pairs]
            if existing_window_sizes and not all(size == window_size for size in existing_window_sizes):
                window_size = min(existing_window_sizes)

            for window_idx in range(1, len(iterable)):
                x = iterable[:window_idx]
                if len(x) > window_size:
                    x = x[-window_size:]
                elif len(x) < window_size:
                    x = (PAD,) * (window_size - len(x)) + tuple(x)
                y = iterable[window_idx]
                
                # development, slow check for correctness
                # assert "".join(
                #     (char for char in (tuple(x) + (y,)) if char != PAD)
                # ) in "".join(iterable)

                data_pairs.append((x, y))

            random.shuffle(data_pairs)
            while len(data_pairs) >= batch_size:
                to_yield = data_pairs[:batch_size]
                data_pairs = data_pairs[batch_size:]
                
                # order by sequence lengths
                sorted_to_yield = sorted(to_yield, key=lambda kv: kv[0].count(PAD))
                yield sorted_to_yield, torch.tensor([len(x) - x.count(PAD) for x, _ in sorted_to_yield])

        # To ensure all batches have the same size and no data is left unused, re-sample last file
        if data_pairs:
            assert len(data_pairs) < batch_size
            cleanup_data_pairs = []
            for window_idx in range(1, len(iterable)):
                x = iterable[:window_idx]
                if len(x) > window_size:
                    x = x[-window_size:]
                elif len(x) < window_size:
                    x = (PAD,) * (window_size - len(x)) + tuple(x)
                y = iterable[window_idx]
                cleanup_data_pairs.append((x, y))
            cleanup_data_pairs = random.sample(cleanup_data_pairs, batch_size - len(data_pairs))
            data_pairs.extend(cleanup_data_pairs)
            # development, check for correctness
            assert len(data_pairs) == batch_size
            data_pairs.sort(key=lambda kv: kv[0].count(PAD))
            yield data_pairs, torch.tensor([len(x) - x.count(PAD) for x, _ in data_pairs])
                
        if not gen_forever:
            break

Here are the first 3 generator outputs, given a batch size of 2 and a maximum window size of 10.

In [6]:
batch_size = 2
max_window_size = 10

gen = batch_sliding_window_generator(training_data, batch_size, max_window_size)
for i in range(3):
    batch, _ = next(gen)
    print("Batch {}".format(i))
    print(*batch, sep="\n")
    print("~"*25)

print("Padding Example")
searching = True
while searching:
    batch, _ = next(gen)
    for x, y in batch:
        if PAD in x:
            searching = False
            break
print((x, y,))

Batch 0
([' ', '=', '=', ' ', 'y', '.', 's', 'h', 'a', 'p'], 'e')
(['s', 'e', 't', '_', 'f', 'i', 'x', 'e', 'd', '_'], 't')
~~~~~~~~~~~~~~~~~~~~~~~~~
Batch 1
(['e', ' ', 'n', 'e', 't', 'w', 'o', 'r', 'k', ' '], 'a')
(['r', ' ', 'g', 'e', 'n', 'e', 'r', 'a', 't', 'i'], 'n')
~~~~~~~~~~~~~~~~~~~~~~~~~
Batch 2
([']', ':', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' '], ' ')
([' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'C', 'r'], 'e')
~~~~~~~~~~~~~~~~~~~~~~~~~
Padding Example
(('\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x02', '"'), '"')


We can see how changing the `batch_size` and `max_window_size` parameters of the generator influence the values we use to train our neural network.

Setting `batch_size=1` and `max_window_size=None` is most computationally expensive option, which has equivalent functionality to the batching algorithm defined in `char_rnn_one_file_code_gen.ipynb`.

In [7]:
# ensure that the dynamic batch window size logic works for arbitrary sequence lengths

# This takes some time to run. Set this to 0 if you understand the batching generator functionality
check_batch_size = 0
# check_batch_size = 1
# check_batch_size = 100

# If set to None, window sizes will default to the number of characters in the file
# check_window_size = 10
check_window_size = None

if check_batch_size:
    print("Checking batch_size={}, max_window_size={}:".format(check_batch_size, check_window_size))
    gen = batch_sliding_window_generator(
        training_data, check_batch_size, max_window_size=check_window_size, gen_forever=False)

    batch_windows = {}
    for batch, _ in gen:
        x, _ = batch[0]
        batch_window_size = len(x)
        assert all(batch_window_size == len(x) for x, _ in batch)
        batch_windows[batch_window_size] = batch_windows.get(batch_window_size, 0) + 1
    print(" • {} number of batches".format(sum(batch_windows.values())))
    for window_size, num_batches in sorted(batch_windows.items(), key=lambda kv: kv[1]):
        print("   • {} batches have window size {}".format(num_batches, window_size))

An additional tensor containing the original lengths of all the padded sequences is also outputted for each batch. It is sorted by length in decreasing order, as per the [pad_packed_sequence](https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.pack_padded_sequence) documentation.

In [8]:
check_batch_size = 10
check_window_size = None
gen = batch_sliding_window_generator(
    training_data, check_batch_size, max_window_size=check_window_size, gen_forever=False)

batch, x_seqs_len = next(gen)
x_seqs, _ = zip(*batch)

for idx, x_seq in enumerate(x_seqs):
    x_seq_len = x_seqs_len[idx].item()
    print("X_Sequence {} has length {} ({} pad chars)".format(idx, x_seq_len, len(x_seq) - x_seq_len))

num_pads = len(x_seqs[0]) - x_seqs_len[0].item()
# print(x_seqs[0][num_pads:][:3])


X_Sequence 0 has length 5757 (1552 pad chars)
X_Sequence 1 has length 5595 (1714 pad chars)
X_Sequence 2 has length 5553 (1756 pad chars)
X_Sequence 3 has length 4713 (2596 pad chars)
X_Sequence 4 has length 3359 (3950 pad chars)
X_Sequence 5 has length 2545 (4764 pad chars)
X_Sequence 6 has length 2101 (5208 pad chars)
X_Sequence 7 has length 577 (6732 pad chars)
X_Sequence 8 has length 256 (7053 pad chars)
X_Sequence 9 has length 214 (7095 pad chars)


## Batch Characters to Tensors

Our batch of inputs and outputs must be converted into Tensors for training.

We follow the default matrix convention used by the PyTorch comunity.

> Tensor’s data will be of size `T x B x *`, where `T` is the length of the longest sequence and `B` is the batch size. 

To make a batch training example, we join a bunch our sequences of one-hot characters into a matrix of size `(window_size, batch_size, num_chars)`.

In [9]:
def batch_xy_to_tensor_xy(batch, num_chars=len(CHARACTERS)):
    batch_size = len(batch)
    window_size = max([len(x) for x, _ in batch])
    assert all(len(x) == window_size for x, _ in batch)
    
    x_tensor = torch.zeros(window_size, batch_size, num_chars)
    y_tensor = torch.zeros(1, batch_size, num_chars)

    for batch_elm_idx, xy_pair in enumerate(batch):
        x_char_seq, y_char = xy_pair
        for seq_idx, x_char in enumerate(x_char_seq):
            x_tensor[seq_idx][batch_elm_idx][CHAR2INT[x_char]] = 1
        y_tensor[0][batch_elm_idx][CHAR2INT[y_char]] = 1
    return x_tensor, y_tensor

The tensor inputs and outputs for the generator function we defined earlier are shown here.

In [10]:
gen = batch_sliding_window_generator(training_data, batch_size, max_window_size)

batch, _ = next(gen)
x = [item[0] for item in batch]
y = [repr(item[1]) for item in batch]
x_tensor, y_tensor = batch_xy_to_tensor_xy(batch)

print("batch_size: {}, max_window_size: {}".format(batch_size, max_window_size))
print("~" * 25)
print(*x, sep="\n")
print(x_tensor.size())
print(*y, sep="\n")
print(y_tensor.size())

batch_size: 2, max_window_size: 10
~~~~~~~~~~~~~~~~~~~~~~~~~
[')', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['t', 'e', ':', ':', '\n', ' ', ' ', ' ', ' ', ' ']
torch.Size([10, 2, 100])
'A'
' '
torch.Size([1, 2, 100])


## Creating the Network

We are going to create our recurrent neural network. Rather than using our own pure recurrent neural network defined in the last notebook, we will be using the [recurrent layers](https://pytorch.org/docs/stable/nn.html#recurrent-layers) provided by PyTorch.
* **RNN**: [Recurrent Neural Network](https://pytorch.org/docs/stable/nn.html#rnn)
* **GRU**: [Gated Recurrent Unit](https://pytorch.org/docs/stable/nn.html#gru)
* **LSTM**: [Long Short-Term Memory](https://pytorch.org/docs/stable/nn.html#lstm)

There are some subtleties in the forward pass function, as we want to mask away the `PAD` characters so they do not influence our model results.

In [14]:
class CharRNN(nn.Module):
    def __init__(self, input_size, output_size, batch_size,
                 hidden_size=128, recurrent_layer_type="RNN", recurrent_layers=1, recurrent_dropout=0):
        super(CharRNN, self).__init__()

        self.hidden_size = hidden_size
        self.recurrent_num_layers = recurrent_layers
        self.batch_size = batch_size

        rn_kwargs = {
            "input_size": input_size,
            "hidden_size": hidden_size,
            "num_layers": recurrent_layers,
            "dropout": recurrent_dropout,
        }
        
        if recurrent_layer_type == "RNN":
            self.input_to_rn = nn.RNN(**rn_kwargs)
        elif recurrent_layer_type == "LSTM":
            self.input_to_rn = nn.LSTM(**rn_kwargs)
        elif recurrent_layer_type == "GRU":
            self.input_to_rn = nn.GRU(**rn_kwargs)
        else:
            raise "Invalid recurrent layer type: {}".format(recurrent_layer_type)

        self.rn_to_output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input_seqs, input_seqs_len, hidden):
        seq_len, batch_size, _ = input_seqs.size()
        
        # pack > recurrent network > unpack
        packed_input_seqs = nn.utils.rnn.pack_padded_sequence(input_seqs, input_seqs_len)
        rn_outs, hidden = self.input_to_rn(packed_input_seqs, hidden)
        unpacked_rn_outs, _ = nn.utils.rnn.pad_packed_sequence(rn_outs, padding_value=CHAR2INT[PAD])

        # send rn output through dense linear layer with softmax activation
        pre_activated_output = self.rn_to_output(unpacked_rn_outs)
        output = self.softmax(pre_activated_output)

        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(self.recurrent_num_layers, self.batch_size, self.hidden_size)

Initialize this network with values appropriate for the character prediction task.

In [15]:
n_chars = len(CHARACTERS)
batch_size = 10

char_rnn = CharRNN(n_chars, n_chars, batch_size)

Before each batch, we zero out the RNN hidden state.

In [17]:
max_window_size = 100
gen = batch_sliding_window_generator(training_data, batch_size, max_window_size, gen_forever=False)
batch, x_seqs_len = next(gen)

x_tensor, y_tensor = batch_xy_to_tensor_xy(batch)


hidden = char_rnn.init_hidden()
output, next_hidden = char_rnn(x_tensor, x_seqs_len, hidden)
print(output.size())
print(hidden.size())

torch.Size([100, 10, 100])
torch.Size([1, 10, 128])
